# Logistic Regression

Refs:
Apply feature selection is a mandatory stuff for logistic regression, via PCA or Variace Threshold and others, worths the reading: 
https://medium.com/@data.science.enthusiast/feature-selection-techniques-forward-backward-wrapper-selection-9587f3c70cfa

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer,KNNImputer
import itertools

In [3]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [4]:
def feature_inputer(
    df:pd.DataFrame,
    to_mode = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],
    to_median = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
) -> pd.DataFrame:
    for col in to_mode:
        mode_inputer = SimpleImputer(strategy='most_frequent')
        df[col] = mode_inputer.fit_transform(df[[col]])

    
    for col in to_median:
        inputer = SimpleImputer(strategy='median')
        df[col] = inputer.fit_transform(df[[col]])
    return df

def dtype_memory_reducer(df: pd.DataFrame) -> pd.DataFrame:
    df['CryoSleep'] = df['CryoSleep'].astype(bool)
    df['VIP'] = df['VIP'].astype(bool)

    df['HomePlanet'] = df['HomePlanet'].astype('category')
    df['Destination'] = df['Destination'].astype('category')
    return df

def outliers_to_log(df:pd.DataFrame) -> pd.DataFrame:
    to_log = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in to_log:
        df[col] = np.log(df[col] + 1)
    return df

def cabin_inputer(df:pd.DataFrame) -> pd.DataFrame:
    cabin_features = df['Cabin'].str.split("/",expand=True)[[0,2]].rename(columns={0:'Deck',2:'side'})
    df = pd.concat([df,cabin_features],axis=1)
    df['Deck'] = df['Deck'].fillna('G').replace("T","G")
    df = df.drop(['side','Cabin'],axis=1)
    return df

def vip_knn_input(df: pd.DataFrame) -> pd.DataFrame:
    inputer = KNNImputer(n_neighbors=5)
    df['VIP'] = inputer.fit_transform(df[['VIP','RoomService']])[:,0]
    return df

def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = cabin_inputer(df)
    df = feature_inputer(df)
    df = vip_knn_input(df)
    df['0_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0
    df = outliers_to_log(df)
    df = dtype_memory_reducer(df)
    return df

def apply_interactions(df:pd.DataFrame, target_col: str):
    for x in itertools.combinations(df.columns.drop(target_col), 2):
        df[f'{x[0]}_{x[1]}'] = df[x[0]]*df[x[1]]
    df = df.drop(df.columns[df.nunique() == 1].tolist(),axis=1)
    return df


def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df,columns=['HomePlanet','Destination','Deck'],drop_first=True)
    return df

In [5]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)

In [6]:
df_train = apply_interactions(df_train, 'Transported')

C:\Users\gonca\AppData\Local\Temp\ipykernel_18320\1635675782.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{x[0]}_{x[1]}'] = df[x[0]]*df[x[1]]
C:\Users\gonca\AppData\Local\Temp\ipykernel_18320\1635675782.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{x[0]}_{x[1]}'] = df[x[0]]*df[x[1]]
C:\Users\gonca\AppData\Local\Temp\ipykernel_18320\1635675782.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

In [7]:
from sklearn.feature_selection import VarianceThreshold
variance = VarianceThreshold(threshold = (.95 * (1 - .95)))
variance.fit(df_train)

VarianceThreshold(threshold=0.04750000000000004)

In [8]:
to_drop = df_train.columns[~variance.get_support()].tolist()
df_train = df_train.drop(to_drop, axis=1)

In [9]:
df_train.head(2)

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,0_bills,HomePlanet_Europa,...,HomePlanet_Europa_Destination_TRAPPIST-1e,HomePlanet_Europa_Deck_B,HomePlanet_Europa_Deck_C,HomePlanet_Mars_Destination_TRAPPIST-1e,HomePlanet_Mars_Deck_F,Destination_PSO J318.5-22_Deck_G,Destination_TRAPPIST-1e_Deck_B,Destination_TRAPPIST-1e_Deck_E,Destination_TRAPPIST-1e_Deck_F,Destination_TRAPPIST-1e_Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,False,39.0,0.00000,0.000000,0.000000,0.000000,0.000000,False,True,1,...,1,1,0,0,0,0,1,0,0,0
0002_01,False,24.0,4.70048,2.302585,3.258097,6.309918,3.806662,True,False,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
X_train,y_train = df_train.drop('Transported',axis=1), df_train['Transported']

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

## Evaluation

In [12]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000)

In [13]:
scores = cross_validate(
    model,
    X_train,
    y_train,
    cv=KFold(5, shuffle=True, random_state=42),
    scoring = "accuracy",
    return_train_score=True
)

In [14]:
scores

{'fit_time': array([0.38994193, 0.32000089, 0.39300203, 0.3319993 , 0.30999851]),
 'score_time': array([0.00300097, 0.00200009, 0.00299859, 0.00299931, 0.0019989 ]),
 'test_score': array([0.77458309, 0.77630822, 0.78723404, 0.78711162, 0.79919448]),
 'train_score': array([0.79666379, 0.79695139, 0.79263733, 0.79338605, 0.79094177])}

In [15]:
results = {
    'train_score_avg':np.mean(scores['train_score']),
    'train_score_std':np.std(scores['train_score']),
    'test_score_avg':np.mean(scores['test_score']),
    'test_score_std':np.std(scores['test_score']),
}
results

{'train_score_avg': 0.7941160676496488,
 'train_score_std': 0.0023377652131206807,
 'test_score_avg': 0.7848862916732564,
 'test_score_std': 0.008887376780059388}

# Submission

In [51]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,
0013_01,True,27.0,False,0.0,0.000000,0.0,0.00000,0.0,True,0,0,0,1,0,0,0,0,0,1
0018_01,False,19.0,False,0.0,2.302585,0.0,7.94591,0.0,False,0,0,0,1,0,0,0,0,1,0


In [52]:
y_test = model.predict(X_test)

In [53]:
submission = pd.Series(y_test, index=[X_test.index]).to_frame('Transported')
submission.to_csv('data/submission_rnforest_2022-11-02.csv')